In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

from src.models.train_model import train_svm
from src.data.dataset_loaders import VehiculesDatasetLoader
from src.features.pipelines import RawImageToFeatures
from src.data.image_loaders import VehiculesFixedSizeImageLoader
from src.models.detectors import SliderProbDetector
from src.data.sliders import SlidingWindow
from src.features.descriptors import HOGDescriptor, LBPDescriptor
from src.models.classifiers import ProbSVMModelWrapper
from src.evaluation import validate_model, precision, recall, f1_score

# Vehicules
# SVM Experiment

### Data loading

In [2]:
DATA_PATH = '../../../data/processed/vehicules/svm/data_svm.pickle'

In [3]:
with open(DATA_PATH, 'rb') as f:
    data = pickle.load(f)

### SVM trainig

In [2]:
SAVE_MODEL_PATH = '../../../models/svm/vehicules_svm.pickle'

In [5]:
model, report = train_svm(data, test_size=0.2, probability=True)

In [6]:
print(report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       961
           1       0.99      0.99      0.99       923

    accuracy                           0.99      1884
   macro avg       0.99      0.99      0.99      1884
weighted avg       0.99      0.99      0.99      1884



In [7]:
with open(SAVE_MODEL_PATH, 'wb') as f:
    pickle.dump(model, f)

### Validation

In [3]:
with open(SAVE_MODEL_PATH, 'rb') as f:
    model = pickle.load(f)

In [9]:
del data

In [4]:
BND_BOX_SIZE = (50, 50)

dataset_loader = VehiculesDatasetLoader(
    image_loader=VehiculesFixedSizeImageLoader(
        bnd_box_size=BND_BOX_SIZE
    )
)

process_pipeline = RawImageToFeatures(
    processors=[],
    descriptors=[
        HOGDescriptor(
            orientations = 9,
            cells_per_block = (2, 2),
            pixels_per_cell = (4, 4),
            multichannel = True,
            visualize = True
        ),
        LBPDescriptor(
            bins = 256,
            range = (0, 256)
        )
    ]
)

sliding_window = SlidingWindow(
    step_size=20,
    window_size=BND_BOX_SIZE
)

#### treshold = 0.7; nms_overlap=0.4

In [5]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ProbSVMModelWrapper(model),
    treshold=0.7,
    nms_overlap=0.4
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath='../../../data/raw/vehicules/val',
    output_folder_filepath='../../processed_images/svm/vehicules/t7nms4',
    images_files_types=('png',),
    annotations_files_types=('txt',),
    detector=slider_detector,
    workers=8
)

print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

No file ../../../data/raw/vehicules/val/00001185.txt
No file ../../../data/raw/vehicules/val/00001143.txt
No file ../../../data/raw/vehicules/val/00001248.txt
No file ../../../data/raw/vehicules/val/00001145.txt
No file ../../../data/raw/vehicules/val/00001244.txt
100%|██████████| 66/66 [36:28<00:00, 33.16s/it]3


Precision: 0.102
Recall: 0.459
F1Score: 0.166


#### treshold = 0.8; nms_overlap=0.4

In [6]:
del processed_images

In [7]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ProbSVMModelWrapper(model),
    treshold=0.8,
    nms_overlap=0.4
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath='../../../data/raw/vehicules/val',
    output_folder_filepath='../../processed_images/svm/vehicules/t8nms4',
    images_files_types=('png',),
    annotations_files_types=('txt',),
    detector=slider_detector,
    workers=8
)

print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

No file ../../../data/raw/vehicules/val/00001185.txt
No file ../../../data/raw/vehicules/val/00001143.txt
No file ../../../data/raw/vehicules/val/00001248.txt
No file ../../../data/raw/vehicules/val/00001145.txt
No file ../../../data/raw/vehicules/val/00001244.txt
100%|██████████| 66/66 [36:06<00:00, 32.82s/it]2


Precision: 0.128
Recall: 0.447
F1Score: 0.199


#### treshold = 0.9; nms_overlap=0.4

In [8]:
del processed_images

In [9]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ProbSVMModelWrapper(model),
    treshold=0.9,
    nms_overlap=0.4
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath='../../../data/raw/vehicules/val',
    output_folder_filepath='../../processed_images/svm/vehicules/t9nms4',
    images_files_types=('png',),
    annotations_files_types=('txt',),
    detector=slider_detector,
    workers=8
)

print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

No file ../../../data/raw/vehicules/val/00001185.txt
No file ../../../data/raw/vehicules/val/00001143.txt
No file ../../../data/raw/vehicules/val/00001248.txt
No file ../../../data/raw/vehicules/val/00001145.txt
No file ../../../data/raw/vehicules/val/00001244.txt
100%|██████████| 66/66 [35:11<00:00, 31.99s/it]1


Precision: 0.163
Recall: 0.435
F1Score: 0.237


#### treshold = 0.95; nms_overlap=0.4

In [14]:
del processed_images

In [15]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ProbSVMModelWrapper(model),
    treshold=0.95,
    nms_overlap=0.4
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath='../../../data/raw/vehicules/val',
    output_folder_filepath='../../processed_images/svm/vehicules/t95nms4',
    images_files_types=('png',),
    annotations_files_types=('txt',),
    detector=slider_detector,
    workers=8
)

print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

No file ../../../data/raw/vehicules/val/00001185.txt
No file ../../../data/raw/vehicules/val/00001143.txt
No file ../../../data/raw/vehicules/val/00001248.txt
No file ../../../data/raw/vehicules/val/00001145.txt
No file ../../../data/raw/vehicules/val/00001244.txt
100%|██████████| 66/66 [35:00<00:00, 31.82s/it]0


Precision: 0.193
Recall: 0.365
F1Score: 0.252


#### treshold = 0.9; nms_overlap=0.3

In [10]:
del processed_images

In [11]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ProbSVMModelWrapper(model),
    treshold=0.9,
    nms_overlap=0.3
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath='../../../data/raw/vehicules/val',
    output_folder_filepath='../../processed_images/svm/vehicules/t9nms3',
    images_files_types=('png',),
    annotations_files_types=('txt',),
    detector=slider_detector,
    workers=8
)

print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

No file ../../../data/raw/vehicules/val/00001185.txt
No file ../../../data/raw/vehicules/val/00001143.txt
No file ../../../data/raw/vehicules/val/00001248.txt
No file ../../../data/raw/vehicules/val/00001145.txt
No file ../../../data/raw/vehicules/val/00001244.txt
100%|██████████| 66/66 [35:12<00:00, 32.01s/it]1


Precision: 0.164
Recall: 0.435
F1Score: 0.239


treshold = 0.9; nms_overlap=0.5

In [12]:
del processed_images

In [13]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ProbSVMModelWrapper(model),
    treshold=0.9,
    nms_overlap=0.5
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath='../../../data/raw/vehicules/val',
    output_folder_filepath='../../processed_images/svm/vehicules/t9nms5',
    images_files_types=('png',),
    annotations_files_types=('txt',),
    detector=slider_detector,
    workers=8
)

print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

No file ../../../data/raw/vehicules/val/00001185.txt
No file ../../../data/raw/vehicules/val/00001143.txt
No file ../../../data/raw/vehicules/val/00001248.txt
No file ../../../data/raw/vehicules/val/00001145.txt
No file ../../../data/raw/vehicules/val/00001244.txt
100%|██████████| 66/66 [35:16<00:00, 32.07s/it]1


Precision: 0.163
Recall: 0.435
F1Score: 0.237
